In [1520]:
import pandas as pd
import numpy as np
import math

# I. Numereisation

## 1. Recuperation de donnee

In [1521]:
def load_data_from_csv(csv_path:str):
    data = pd.read_csv(csv_path)
    return data

## 2. Encodage

In [1522]:
def encodage(data,indexes:list,encode_rules):
    ''' Permet de faire l'encodage des donnees 
        ARGS
            - data : la liste des donnees
            - indexes : la listes des colonnes a encoder
            - encode_rules : un dictionaire des regles a suivre pour encodes la valeur des donnees
    '''
    for i in range(len(data)):
        for index in indexes:
            value = data.loc[i,index]
            try:
                value = encode_rules[index][value]
            except :
                value = encode_rules[value]
            data.loc[i,index] = value

# II. Entropie

In [1523]:
def entopie_piece(repetion:int,total:int):
    result = repetion / total
    return result

In [1524]:
def entropie(data:pd.DataFrame,classes):
    ''' Etablir l'entropie a partir d'une colonne de classe 
        PARAMETERS :
            - data : les donnees pour etablir l'entropie
            - class_attribute : Le nom de la colonne qui represente les donnee pour l'entropie
        RETURN :
            - entropie , la valeur entropique des donnees par rapport au colonne donnee
    '''
    # Le nombre total de donnee
    data_lenght = len(data)
    # Recuperer les valeurs de la classe
    couple_vlaues = data[classes]
    # Calculer la repetions de chacunes des valeurs parmi les donnees
    repetitions = couple_vlaues.groupby(classes).size().reset_index(name='Repetitions')
    # Le resultat attendue
    entropie = 0
    # Calcul de l'entropie
    for rep in repetitions['Repetitions']:
        # La piece entropique de la valeur
        pc = entopie_piece(rep,data_lenght)
        # Incrementation de la somme
        it = -(pc) * math.log2(pc)
        entropie += it
    return entropie

# III. Meilleure Caracteristique

## 1. Gain

### 1.1. Separation de donnee

In [1525]:
def split_data(dataframe,colonne):
    """
    Sépare les données d'un DataFrame selon une colonne spécifique en regroupant les mêmes critères.

    Parameters:
    dataframe (pd.DataFrame): Le DataFrame contenant les données.
    colonne (str): Le nom de la colonne selon laquelle séparer les données.

    Returns:
    dict: Un dictionnaire où les clés sont les valeurs uniques de la colonne et les valeurs sont les DataFrames correspondants.
    """
    groupes = dataframe.groupby(colonne)
    resultats = {nom: groupe for nom , groupe in groupes}
    return resultats

### 1.2. Fonction de gain

In [1526]:
def gain(data:pd.DataFrame,ref_attribute:str,classes):
    # Entropie de l'ensemble du data
    entropie_S = entropie(data,classes)
    # Le gain attendue
    gain = entropie_S
    # Le nombre de donnee total
    nS = len(data)
    # Separation des donnees en plusieurs sous section de donnee
    data_sv = split_data(data,ref_attribute)
    for index, data_part in data_sv.items():
        # Le nombre de sous section
        nSv = len(data_part)
        sv = nSv/nS
        # Entropie du sous section
        entropie_SV = entropie(data_part,classes)

        gain -= sv*entropie_SV
    return gain

## 2. Calculs des GAINS

In [1527]:
def gains(data:pd.DataFrame,classes):
    ''' 
        Calcul des gains pour chaque critere present parmi les donnees
        ARGS :
            - data : la liste des donnees d'entrainement
            - classes : le nom des colonnes qui prescisent la class des donees
        RETURNS :
            - gains , tableau des couples ( nom critere , gain )
    '''
    # Resulats > Liste des gains
    gains = list()
    # Recuperation des criteres possibles
    indexes = data.columns
    for index in indexes :
        if index in classes:
            continue
        gains.append({index : gain(data,index,classes)})
    return gains

## 3. Best collumn

In [1528]:
def find_best_criteria(data:pd.DataFrame,classes):
    '''
        Calcul le gain d'information de chaque critere de donnee pour determiner le meilleur parmi eux
        ARGS :
            - data : la liste des donnees d'entrainement
            - classes : le nom de la colonne qui prescise la class des donees
        RETURNS :
            - criteria , le nom du critere qui possede le gain la plus haute apres filtre decroissante
    '''
    # Recuperation des gains pour chaque criteres
    gains_var = gains(data,classes)
    # Filtrer decroissant
    gains_sorted = sorted(gains_var, key=lambda d : list(d.values())[0])
    gains_sorted.reverse()
    return [criteria for criteria , gain_value in gains_sorted[0].items()][0]

# IV. ARBRE

## 1. Creation d'un arbre

### 1.1. Creation d'une branche

In [1529]:
def generate_tree_branch(splited_data,classes):
    branch = {}
    for key in splited_data.keys():
        # Data de chaque section
        data_part = splited_data[key]
        # On verifie l'entropie 
        data_entropie = entropie(data_part,classes)
        if data_entropie == 0:
            # Pour une entropie homogene
            branch[key] = np.array(data_part[classes].drop_duplicates().reset_index(drop=True))[0]
        else :
            branch[key] = generate_tree_body(data_part,classes)
    return branch

### 1.2. Creation d'un tronc

In [1530]:
def generate_tree_body(data:pd.DataFrame,classes):
    # Recuperer le meilleur critre
    best_criteria = find_best_criteria(data,classes)
    # The tree decision
    tree_body = {
        'body_name':best_criteria
    }
    # Diviser en sous section des donnees du best criteria
    splited_data = split_data(data,best_criteria)
    # Generate the branches of the body
    tree_body['branch'] = generate_tree_branch(splited_data,classes)
    return tree_body

### 1.3. Tree gen

In [1531]:
def generate_tree(data:pd.DataFrame,classes):
    # The tree decision
    tree = {
        'class_name':classes,
        'tree_body':generate_tree_body(data,classes)
    }
    return tree

## 2. Prise de decision

In [1532]:

def get_tree_body_class_result(data:object,tree_body:dict,class_count=1):
    body_name = tree_body['body_name']
    branch  = tree_body['branch']
    # Recuperer le chemin logique du donnee dans l'arbre
    branch_path = data[body_name]
    try :
        class_result = branch[branch_path]
        if type(class_result) ==  dict:
            class_result = get_tree_body_class_result(data,class_result)
    except :
        class_result = [None] * class_count
    return class_result

In [1533]:
def get_class_by_tree(data:object,tree:dict):
    # La class a determiner
    data_class = None
    tree_body = tree['tree_body']
    data_class = get_tree_body_class_result(data,tree_body,len(tree['class_name']))
    return data_class

# V. FOREST

## 1. Shuffle Data

In [1534]:
def shuffle_data(dataframe:pd.DataFrame):
    """
        Retourne une copie mélangée d'un DataFrame.

        Parameters:
        dataframe (pd.DataFrame): Le DataFrame contenant les données.

        Returns:
        pd.DataFrame: Une copie mélangée du DataFrame.
    """
    # Mélanger les lignes du DataFrame
    dataframe_melange = dataframe.sample(frac=1, random_state=1).reset_index(drop=True)
    return dataframe_melange

## 2. Data select 

In [1535]:
def data_select(dataframe:pd.DataFrame, pourcentage:int):
    """
    Récupère un pourcentage donné de lignes d'un DataFrame.

    Parameters:
    dataframe (pd.DataFrame): Le DataFrame contenant les données.
    pourcentage (float): Le pourcentage de lignes à récupérer (entre 0 et 100).

    Returns:
    pd.DataFrame: Un DataFrame contenant le pourcentage spécifié de lignes.
    """
    if pourcentage < 0 or pourcentage > 100:
        raise ValueError("Le pourcentage doit être entre 0 et 100.")
    
    # Calcul du nombre de lignes à récupérer
    nombre_lignes = int(len(dataframe) * (pourcentage / 100))
    
    # Sélection aléatoire des lignes
    echantillon = dataframe.sample(nombre_lignes)
    
    return echantillon

## 3. Random data select

In [1536]:
def random_data_select(data:pd.DataFrame,pourcentage:int):
    shuffled_data = shuffle_data(data)
    echantillon = data_select(shuffled_data,pourcentage)
    return echantillon

## 4. Forest gen

In [1537]:
def generate_forest(train_data:pd.DataFrame,classes,pourcentage:int,tree_number:int):
    forest = {
        'classes' : classes
    }
    forest_entity = []
    for i in range(tree_number):
        echantillon = random_data_select(train_data,pourcentage)
        tree = generate_tree(echantillon,classes)
        forest_entity.append(tree)
    forest['entity'] = forest_entity
    return forest

## 5. Forest decision

In [1538]:
def compter_tous_les_couples(liste_couples):
    """
    Compte le nombre de répétitions de chaque couple dans une liste de couples.

    Parameters:
    liste_couples (list): Une liste de couples (chaque couple est représenté par une liste ou un tuple).

    Returns:
    dict: Un dictionnaire où les clés sont les couples et les valeurs sont le nombre de répétitions.
    """
    compteur_couples = {}
    for couple in liste_couples:
        couple_tuple = tuple(couple)  # Convertir le couple en tuple pour pouvoir l'utiliser comme clé de dictionnaire
        if couple_tuple in compteur_couples:
            compteur_couples[couple_tuple] += 1
        else:
            compteur_couples[couple_tuple] = 1
    return compteur_couples

In [1539]:
def get_class_by_forest(data:pd.DataFrame,forest):
    data_class = None
    
    tree_decisions = []
    for tree in forest['entity']:
        tmp_class = get_class_by_tree(data,tree)
        # if tmp_class in tree_decisions.keys():
        #     tree_decisions[tmp_class] += 1
        # else:
            # tree_decisions[tmp_class] = 1
        tree_decisions.append(tmp_class)
    couple_count = compter_tous_les_couples(tree_decisions)
    max_count = 0
    for clazz in couple_count.keys():
        local_class_count = couple_count[clazz]
        if max_count < local_class_count:
            data_class = clazz
            max_count = local_class_count
    
    return pd.DataFrame(data=data_class,index=forest['classes']).transpose()
    # return data_class

# Main

## MAIN FOREST

In [1540]:
# Recuperation des donnees
data = load_data_from_csv("data/spam.csv")
pourcentage = 20
tree_number = 10
class_attribute = ['Classe']
# Encodage des donnees
# encodage(
#     data,
#     indexes=['Previsions','Temperature','Humidite','Vent','Classe'],
#     encode_rules={
#         'Ensoleille':1,'Nuageux':2,'Pluvieux':3,
#         'Chaud':3,'Moyen':2,'Frais':1,
#         'Elevee':2,'Normale':1,
#         'Faible':1,'Fort':2,
#         'Oui':0,'Non':1
#     }
# )
forest = generate_forest(data,class_attribute,pourcentage,tree_number)
# print(forest)
get_class_by_forest(data.loc[1],forest)

,Classe
0,Oui


In [1541]:
# Recuperation des donnees
data = load_data_from_csv("data/spam_copy.csv")
pourcentage = 20
tree_number = 1
class_attribute = ['Classe','Periode']
# Encodage des donnees
# encodage(
#     data,
#     indexes=['Previsions','Temperature','Humidite','Vent','Classe'],
#     encode_rules={
#         'Ensoleille':1,'Nuageux':2,'Pluvieux':3,
#         'Chaud':3,'Moyen':2,'Frais':1,
#         'Elevee':2,'Normale':1,
#         'Faible':1,'Fort':2,
#         'Oui':0,'Non':1
#     }
# )
forest = generate_forest(data,class_attribute,pourcentage,tree_number)
# print(forest)
get_class_by_forest(data.loc[1],forest)

,Classe,Periode
0,Non,Soiree


## MAIN TREE

In [1542]:
# Recuperation des donnees
data = load_data_from_csv("data/spam.csv")
# Encodage des donnees
# encodage (
#     data,
#     indexes=['Previsions','Temperature','Humidite','Vent','Classe'],
#     encode_rules={
#         'Ensoleille':1,'Nuageux':2,'Pluvieux':3,
#         'Chaud':3,'Moyen':2,'Frais':1,
#         'Elevee':2,'Normale':1,
#         'Faible':1,'Fort':2,
#         'Oui':0,'Non':1
#     }
# )
# Creation d'un arbre
tree = generate_tree(data,['Classe'])
# Classer a partir d'un arbre
print(tree)
for i in range(len(data)):    
    classification = get_class_by_tree(data.loc[i],tree)
    print(classification)

{'class_name': ['Classe'], 'tree_body': {'body_name': 'Previsions', 'branch': {'Ensoleille': {'body_name': 'Humidite', 'branch': {'Elevee': array(['Non'], dtype=object), 'Normale': array(['Oui'], dtype=object)}}, 'Nuageux': array(['Oui'], dtype=object), 'Pluvieux': {'body_name': 'Vent', 'branch': {'Faible': array(['Oui'], dtype=object), 'Fort': array(['Non'], dtype=object)}}}}}
['Non']
['Non']
['Oui']
['Oui']
['Oui']
['Non']
['Oui']
['Non']
['Oui']
['Oui']
['Oui']
['Oui']
['Oui']
['Non']


# MULTI 

In [1543]:
data = load_data_from_csv("data/spam.csv")
classes = [
    # 'Humidite'
    # ,'Vent'
    'Classe'
    # ,'Periode'
    # ,'Previsions'
]
print(gains(data,classes))
# ENTROPIE
tree = generate_tree(data,classes)
get_class_by_tree(data.loc[4],tree)

[{'Previsions': 0.24674981977443933}, {'Temperature': 0.02922256565895487}, {'Humidite': 0.15183550136234164}, {'Vent': 0.048127030408269544}]


array(['Oui'], dtype=object)

In [1544]:
# criteres = data[['Humidite','Vent']]
criteres = data[classes]
# set(criteres)
unique_criteres = criteres.drop_duplicates().reset_index(drop=True)
np.array(unique_criteres)
# repetitions = criteres.groupby(classes).size().reset_index(name='Repetitions')

array([['Non'],
       ['Oui']], dtype=object)